# Notebook para ejecutar el proceso de extracción de datos del sítio https://www.espaciourbano.com/

## Nota Importante!!!:
El proceso a continuacion describe los pasos seguidos para la obtención de la información por scraping de un sitio activo que oferta inmuebles en arriendo en la ciudad de Medellín, esto quiere decir que al ejecutarlo en fechas diferentes es posible que no se genere la misma información dadas las dinamicas de este negocio.

**Los datos utilizados para el modelo se encuentran el la carpera _datos_modelo_**

Librerías

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import random
import time

Lectura de archivos en utilidades

In [2]:
# Carga de listado de users agents
user_agets = pd.read_csv("utils/user-agent.csv")

# Carga de listado de proxies
with open("utils/proxies.json") as json_file:
    proxies = json.load(json_file)
proxies = proxies.get("proxies")

**Definiciones de URLs**

El sitio espacio urbano contiene 5 zonas para casas en alquiler en la ciudad de Medellín:
- Centro
- Poblado
- Belen
- Laureles
- San Antonio de Prado

Cada una de las zonas contiene una URL producto de interactuar con el sitio y hacer los respectivos filtros por zonas, además cada una de estas puede tener resultados en n páginas, estas urls se definen a continuación

In [3]:
# URL Principal
head_url = "https://www.espaciourbano.com/"

# URL Zona Centro
urls_zona_centro   = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10000&pTipoInmueble=1&nCiudad=Medellin%20Zona%201%20-%20Centro",          # Página 1
                      "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10000&pTipoInmueble=1&nCiudad=Medellin+Zona+1+%2D+Centro&offset={pages}"] # Página n

# URL Zona Poblado
urls_poblado = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10027&pTipoInmueble=1&nCiudad=Medellin%20Zona%202%20-%20El%20Poblado",         # Página 1
                "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10027&pTipoInmueble=1&nCiudad=Medellin+Zona+2+%2D+El+Poblado&offset={pages}"]  # Página n

# URL Zona Belen
urls_belen = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10029&pTipoInmueble=1&nCiudad=Medellin%20Zona%204%20-%20Belen",          # Página 1
              "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10029&pTipoInmueble=1&nCiudad=Medellin+Zona+4+%2D+Belen&offset={pages}"] # Página n

# URL Laureles
urls_laureles = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10028&pTipoInmueble=1&nCiudad=Medellin%20Zona%203%20-%20Laureles",          # Página 1
                 "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10028&pTipoInmueble=1&nCiudad=Medellin+Zona+3+%2D+Laureles&offset={pages}"] # Página n

# URL San Antonio de Prado
urls_san_antonio = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10041&pTipoInmueble=1&nCiudad=San%20Antonio%20de%20Prado",          # Página 1
                    "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10041&pTipoInmueble=1&nCiudad=San+Antonio+de+Prado&offset={pages}"] # Página n

**Funciones:**

La información de cada uno de los inmuebles esta dividida en
- Precio
- Zona
- Comonidades
- Caracteristicas

Se define una función que realiza la petición a cada sitio y extrae la información de cada inmueble con el respectivo código de registro de la página

In [4]:
def get_properties_info(url: str) -> tuple:
    """
    Función para realizar la petición a una url especifica
    y devolver los datos de cada inmueble

    PARAMS
        url: str
            url de una página con el listado de los inmuenles en arriendo
    RETURN
        (cod, lease) tuple
            código de un inmueble con su respectivo grupos de valores
            precio, zona, comodidades y caracteristicas
    """
    # Variar el user agent por petición
    headers = {'User-Agent': user_agets.sample(1)["user-agent"].values[0]}

    # Consulta y carga de la url de detalle
    detail = requests.get(url.replace("Ficha","ficha"), headers=headers)

    soup_detail =BeautifulSoup(detail.content,'lxml')

    # Obtención del código de vivienda, precio y zona
    price_zone = soup_detail.find_all('div', attrs={'class':'text-center'})
    price_zone = price_zone[1].find_all('h3')
    code = soup_detail.find_all('div', attrs={'class':'text-center'})[0].text.strip()
    price = price_zone[0].text.replace("\r\n","").strip()
    zone = price_zone[2].text

    # Obtención de las comodidades
    comforts = []
    comforts_div = soup_detail.find('div', attrs={'class' : 'col-lg-4'}).find_all('p')
    for com in comforts_div:
        comfort = com.text.strip()
        if comfort != "": 
            comforts.append(comfort)

    
    # Obtención de las características
    characteristics = {}
    characteristics_div = soup_detail.find('div', attrs={'class' : 'col-lg-8'}).find('table').find_all('tr')
    for cha in characteristics_div:
        row_cha = cha.find_all('td')
        characteristics[row_cha[0].text.strip()] = row_cha[1].text.strip()

    lease = {
        "precio" : price,
        "zona" : zone,
        "comodidades" : comforts,
        "caracteristicas" : characteristics
    }

    # retorno de valores
    return code, lease

In [5]:
def write_file(data: set, file_name: str) -> None:
    """
    Función para imprimir datos en un archivo .txt
    PARAMS
        data: set
            Conjunto con los datos a imprimir en el archivo
        file_name: str
            Nombre del archivo a crear
    RETUN
    
    """
    # Transfomación de los datos a una lista
    data = list(data)
    # Crear archivo sobre el cual se va a escribir
    with open(f'{file_name}.txt', 'w') as f:
        # Ciclo para escribir cada línea en el archivo
        for line in data:
            f.write(line + '\n')
    f.close

In [6]:
def get_zone_info(urls: list,
                  pages: int,
                  path: int,
                  file_name: str,
                  zona: str = "") -> None:
    """
    Función para consultar la información de los inmuebles por cada zona
    PARAMS
        urls:lis
            Lista de las urls por zona
        pages: int
            Cantidad de páginas que tiene cada zona
        path: int
            Salto de página de cada sitio
        file_name: str
            Nombre del archivo donde se almacenan los resultados
        zona: str
            Nombre de zona que se esta procesando
    RETURN
        leases: dict
            Información de los inmuebles por zona
        characteristics: set
            Consolidado de los tipos de características por zona
        comforts: set
            Consolidado de las diferentes comodidades por zona
    """

    # Diccionario para almacenar los resultados
    leases = {}
    # Conjuntos para almacenar los consolidados de
    # Comodidades y características
    characteristics = set()
    comforts = set()

    print(f"Consultando Zona: {zona}")

    for i in range(0, pages):
        
        user_agets.sample(1)["user-agent"].values[0]
        # Se selecciona un agente de forma aleatoria
        headers = {'User-Agent': user_agets.sample(1)["user-agent"].values[0]}
        
        # Selección de la página a consultar
        if i == 0:
            url = urls[0]
        else:
            url = urls[1].format(pages = i*path)

        print(f"-- Consultando página : {i + 1}")
        response = requests.get(url,headers=headers)
        
        if response.status_code == 200:
            # Por medio de BeautifulSoup se extrae el contendio de la pagina 
            soup_list = BeautifulSoup(response.content,'lxml')

            # Cargar todas las clases div con para obtener el detalle de las viviendas
            urls_details = soup_list.find_all('div', attrs={'class' : 'col-sm-4'})
            urls_details.pop(0)
            for url_detail in urls_details:
                a = url_detail.find('a')
                url = head_url + a.get('href')
                code, lease = get_properties_info(url)
                leases[code] = lease
                characteristics = characteristics.union(set(lease["caracteristicas"].keys()))
                comforts = comforts.union(set(lease["comodidades"]))
        else:
            print("ERROR: Falla consultando url: {}".format(url))
    
    # Almacenado en archivos
    print("Imprimiendo Resultados...")
    with open(f"{file_name}.json", "w", encoding='utf-8') as outfile:
        json.dump(leases, outfile)
    write_file(characteristics, file_name + "_carac")
    write_file(comforts, file_name + "_com")
    print("Scraping Completo")

***Ejecución de proceso de peticiones o requests:***

Teniendo las urls y las funciones se procede a ejecutar el proceso de extracción de información por cada zona, en este proceso se deben generar 3 archivos:
- .json: archivo json por zona con la informacion de los inmuebles
- .txt: archivo con el consolidado de las características de todos los inmuebles de cada zona
- .txt: archivo con el consolidado de las comodidades de todos los inmuebles de cada zona

Los archivos de comodidades y características se deben generar para crear una variable booleana con cada uno de los valores de estos archivos, por tanto por cada inmueble se tendrá una variable booleana que indique si tiene o no dicha caracteristica

Estos resultados serán almacenados en la carpeta _resultados_request_

**Scraping: Zona Centro**

In [9]:
# Parametros
urls = urls_zona_centro
pages = 5 # Número de páginas del sitio para esta zona
path = 50 # Indica el salto de página
file_name = "resultados_request/zona_1_centro"
characteristics_file = "resultados_request/zona_1_centro_carac"
comforts_file = "resultados_request/zona_1_centro_com"

In [10]:
# Ejecución del scraping
get_zone_info(urls,
              pages,
              path,
              file_name,
              zona = "Centro")

Consultando Zona: Centro
-- Consultando página : 1
-- Consultando página : 2
-- Consultando página : 3
-- Consultando página : 4
-- Consultando página : 5
Imprimiendo Resultados...
Scraping Completo


**Scraping: Zona Poblado**

In [11]:
# Parametros
urls = urls_poblado
pages = 38 # Número de páginas del sitio para esta zona
path = 50 # Indica el salto de página
file_name = "resultados_request/zona_2_poblado"
characteristics_file = "resultados_request/zona_2_poblado_carac"
comforts_file = "resultados_request/zona_2_poblado_com"

In [12]:
# Ejecución del scraping
get_zone_info(urls,
              pages,
              path,
              file_name,
              zona = "Poblado")

Consultando Zona: Poblado
-- Consultando página : 1
-- Consultando página : 2
-- Consultando página : 3
-- Consultando página : 4
-- Consultando página : 5
-- Consultando página : 6
-- Consultando página : 7
-- Consultando página : 8
-- Consultando página : 9
-- Consultando página : 10
-- Consultando página : 11
-- Consultando página : 12
-- Consultando página : 13
-- Consultando página : 14
-- Consultando página : 15
-- Consultando página : 16
-- Consultando página : 17
-- Consultando página : 18
-- Consultando página : 19
-- Consultando página : 20
-- Consultando página : 21
-- Consultando página : 22
-- Consultando página : 23
-- Consultando página : 24
-- Consultando página : 25
-- Consultando página : 26
-- Consultando página : 27
-- Consultando página : 28
-- Consultando página : 29
-- Consultando página : 30
-- Consultando página : 31
-- Consultando página : 32
-- Consultando página : 33
-- Consultando página : 34
-- Consultando página : 35
-- Consultando página : 36
-- Consulta

**Scraping: Zona Belén**

In [13]:
# Parametros
urls = urls_belen
pages = 8 # Número de páginas del sitio para esta zona
path = 50 # Indica el salto de página
file_name = "resultados_request/zona_3_belen"
characteristics_file = "resultados_request/zona_3_belen_carac"
comforts_file = "resultados_request/zona_3_belen_com"

In [14]:
# Ejecución del scraping
get_zone_info(urls,
              pages,
              path,
              file_name,
              zona = "Belén")

Consultando Zona: Belén
-- Consultando página : 1
-- Consultando página : 2
-- Consultando página : 3
-- Consultando página : 4
-- Consultando página : 5
-- Consultando página : 6
-- Consultando página : 7
-- Consultando página : 8
Imprimiendo Resultados...
Scraping Completo


**Scraping: Zona Laureles**

In [39]:
# Parametros
urls = urls_laureles
pages = 19 # Número de páginas del sitio para esta zona
path = 50 # Indica el salto de página
file_name = "resultados_request/zona_4_laureles"
characteristics_file = "resultados_request/zona_4_laureles_carac"
comforts_file = "resultados_request/zona_4_laureles_com"

In [40]:
# Ejecución del scraping
get_zone_info(urls,
              pages,
              path,
              file_name,
              zona = "Laureles")

Consultando Zona: Laureles
-- Consultando página : 1
-- Consultando página : 2
-- Consultando página : 3
-- Consultando página : 4
-- Consultando página : 5
-- Consultando página : 6
-- Consultando página : 7
-- Consultando página : 8
-- Consultando página : 9
-- Consultando página : 10
-- Consultando página : 11
-- Consultando página : 12
-- Consultando página : 13
-- Consultando página : 14
-- Consultando página : 15
-- Consultando página : 16
-- Consultando página : 17
-- Consultando página : 18
-- Consultando página : 19
Imprimiendo Resultados...
Scraping Completo


**Scraping: Zona San Antonio de Prado**

In [17]:
# Parametros
urls = urls_san_antonio
pages = 2 # Número de páginas del sitio para esta zona
path = 50 # Indica el salto de página
file_name = "resultados_request/zona_5_san_antonio"
characteristics_file = "resultados_request/zona_5_san_antonio_carac"
comforts_file = "resultados_request/zona_5_san_antonio_com"

In [22]:
# Ejecución del scraping
get_zone_info(urls,
              pages,
              path,
              file_name,
              zona = "San Antonio de Prado")

Consultando Zona: San Antonio de Prado
-- Consultando página : 1
-- Consultando página : 2
Imprimiendo Resultados...
Scraping Completo


**Procesado de los datos**

Al obtener los resultados del scraping y el consolidado de las caracteristicas y comodidades se deben organizar los datos de manera tabulada para construir la base de entrenamiento del modelo. Este proceso de consiste en:
- Por zona crear un archivo csv cuyas columnas esten formateadas sin acentos ni caracteres especiales que afecten su lectura
- Las variables obtenidas se dividen en 3 grupos
    - general: variables como: precio, area
    - caracteristicas: variables como: tipo pisos, número de niveles
    - comodidades: variables como: zona de ropas, jacuzzi, red de gas
- Por cada caracteristica del consolidado se crea una variable booleana para indicar si el inmueble cuenta o no con esa característica
- La comodidades obtenidas se convierten en variables categoricas numéricas, estas son las que expresan con una cantidad las ventajas de una vivienda, por ejemplo, numero de baños, habitaciones, etc
- El restante de las variables serán numéricas o categóricas

In [36]:
def proccess_data_scraping(data: dict, format_columns: list, columns: dict, file_name: str) -> None:
    """
    Funcion para procesar y tabular los datos obtenidos por el proceso de web scraping
    PARAMS
        data: dict
            Diccionario con los datos obtenidos por el proceso de web scraping
        format_columns: list
            Listado de columnas formateadas
        colums: dict
            Diccionario separado por grupo de variables consolidadas
        file_name: str
            Nombre del archivo donde se almacenan los resultados
            
    """
    data_frame_zona = pd.DataFrame(columns=format_columns)

    viviendas = list(data.keys())
    row = {}
    print("Formateando datos...")
    for vivienda in viviendas:
        dat = data[vivienda]
        # Lectura de variables generales
        row["codigo"] = vivienda
        row["precio"] = dat["precio"]
        row["zona"] = dat["zona"]

        # Variables
        for car in columns["caracteristicas"].keys():
            row[columns["caracteristicas"][car]] = dat["caracteristicas"][car]

        for com in columns["comodidades"].keys():
            if com in dat["comodidades"]:
                row[columns["comodidades"][com]] = [1]
            else:
                row[columns["comodidades"][com]] = [0]
        
        data_frame_zona = pd.concat([data_frame_zona,pd.DataFrame.from_dict(row)])

    data_frame_zona.to_csv(file_name, index=False)
    print("Formato completo")


In [37]:
# Lectura de carchivo con el nombre de las variables ya formateado
format_columns = pd.read_csv("utils/format_columns.csv", encoding='utf-8')
format_columns = list(format_columns["columnas"])

# Lectura de archivo con el nombre de las columnas por cada grupo (general, caracteristicas, comodidades)
# Consolidando los totales
with open("utils/columns.json", encoding="utf-8") as columns:
    columns = json.load(columns)

['codigo', 'precio', 'zona', 'barrio_sector', 'tipo_pisos', 'baños_familiares', 'area_bruta', 'ciudad', 'numero_niveles', 'tipo_cocina', 'parqueaderos', 'otras_comodidades', 'alcobas_familiares', 'estrato', 'area_total', 'juegos_infantiles', 'balcon', 'zona_ropas', 'camaras_cctv', 'cancha_polideportiva', 'ascensor', 'cancha_squash', 'zona_bbq', 'patio', 'unidad_cerrada_conjunto', 'zonas_verdes', 'aire_acondicionado', 'jacuzzi', 'red_de_Gas', 'turco', 'porteria_24_7', 'sauna', 'calentador_de_agua', 'terraza', 'closet_de_linos', 'biblioteca', 'parqueadero_visitantes', 'gimnasio', 'piscina', 'salon_social', 'dispositivos_automatizacion', 'alarma']


**Formateo de datos zona 1 centro**

In [42]:
# Lectura de los datos del scraping
with open("resultados_request/zona_1_centro.json", encoding="utf-8") as data_scraping:
    data = json.load(data_scraping)
proccess_data_scraping(data, format_columns, columns, 'resultados_procesar_datos/data_zona_1.csv')

Formateando datos...
Formato completo


**Formateo de datos zona 2 Poblado**

In [43]:
# Lectura de los datos del scraping
with open("resultados_request/zona_2_poblado.json", encoding="utf-8") as data_scraping:
    data = json.load(data_scraping)
proccess_data_scraping(data, format_columns, columns, 'resultados_procesar_datos/data_zona_2.csv')

Formateando datos...
Formato completo


**Formateo de datos zona 3 Belén**

In [44]:
# Lectura de los datos del scraping
with open("resultados_request/zona_3_belen.json", encoding="utf-8") as data_scraping:
    data = json.load(data_scraping)
proccess_data_scraping(data, format_columns, columns, 'resultados_procesar_datos/data_zona_3.csv')

Formateando datos...
Formato completo


**Formateo de datos zona 4 Laureles**

In [45]:
# Lectura de los datos del scraping
with open("resultados_request/zona_4_laureles.json", encoding="utf-8") as data_scraping:
    data = json.load(data_scraping)
proccess_data_scraping(data, format_columns, columns, 'resultados_procesar_datos/data_zona_4.csv')

Formateando datos...
Formato completo


**Formateo de datos zona 5 San Antonio de Prado**

In [46]:
# Lectura de los datos del scraping
with open("resultados_request/zona_5_san_antonio.json", encoding="utf-8") as data_scraping:
    data = json.load(data_scraping)
proccess_data_scraping(data, format_columns, columns, 'resultados_procesar_datos/data_zona_5.csv')

Formateando datos...
Formato completo
